# Importing Data
- Scraping names and committee names of Senate Committee Members
- Scraping individual data from wikipedia from the previous list created
- Creating dataframes of both committee memebers ranking member and chair 
- including voter data, state, assumed office, time in office

In [1]:
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_United_States_Senate_committees"

In [3]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')
#print(soup.prettify())
names = pd.read_html(url)

In [4]:
#columnNames = smallDF.columns().tolist()
sliceNames = names[0]

In [5]:
# names[0].head()

In [6]:
sliceNames.to_csv('names.csv', index = False)

In [7]:
importNames = pd.read_csv('names.csv')
importNames

,Committee,Committee.1,Chair,Ranking Member,Refs
0,Unnamed: 0_level_1,Subcommittee,Chair,Ranking Member,Refs
1,"Agriculture, Nutrition and Forestry","Agriculture, Nutrition and Forestry",Pat Roberts (R-KS),Debbie Stabenow (D-MI),[1][2]
2,NaN,"Commodities, Risk Management and Trade",John Boozman (R-AR),Sherrod Brown (D-OH),[3]
3,NaN,"Conservation, Forestry and Natural Resources",Mike Braun (R-IN),Michael Bennet (D-CO),[3]
4,NaN,"Livestock, Marketing and Agriculture Security",Cindy Hyde-Smith (R-MS),Kirsten Gillibrand (D-NY),[3]
...,...,...,...,...,...
80,NaN,Intellectual Property,Thom Tillis (R-NC),Chris Coons (D-DE),[14]
81,NaN,"Oversight, Agency Action, Federal Rights and F...",Ben Sasse (R-NE),Richard Blumenthal (D-CT),[14]
82,Rules and Administration,Rules and Administration,Roy Blunt (R-MO),Amy Klobuchar (D-MN),[1][2]
83,Small Business and Entrepreneurship,Small Business and Entrepreneurship,Marco Rubio (R-FL),Ben Cardin (D-MD),[1][2]


In [8]:
committee = importNames['Committee']
chairNames = importNames['Chair']
rankingMembers = importNames['Ranking Member']

In [9]:
chairNames = list(chairNames)
chairNames = chairNames[1:]
rankingMembers = rankingMembers[1:]

In [10]:
#Pulling out list of names from scrape
#chairNames[1][:12]
def stripNames(stripList):
    stateParty = []
    fullName = []
    party = []
    state = []
    for i in stripList:
        counter = 0
        for j in i:
            counter += 1
            if j == '(':
                stateParty.append(i[counter-1:])
                fullName.append(i[:counter-2])
                party.append(i[-5:len(i)-4])
                state.append(i[-3:len(i)-1])
                #print(counter)
    return fullName, stateParty, party, state

In [11]:
importNames

,Committee,Committee.1,Chair,Ranking Member,Refs
0,Unnamed: 0_level_1,Subcommittee,Chair,Ranking Member,Refs
1,"Agriculture, Nutrition and Forestry","Agriculture, Nutrition and Forestry",Pat Roberts (R-KS),Debbie Stabenow (D-MI),[1][2]
2,NaN,"Commodities, Risk Management and Trade",John Boozman (R-AR),Sherrod Brown (D-OH),[3]
3,NaN,"Conservation, Forestry and Natural Resources",Mike Braun (R-IN),Michael Bennet (D-CO),[3]
4,NaN,"Livestock, Marketing and Agriculture Security",Cindy Hyde-Smith (R-MS),Kirsten Gillibrand (D-NY),[3]
...,...,...,...,...,...
80,NaN,Intellectual Property,Thom Tillis (R-NC),Chris Coons (D-DE),[14]
81,NaN,"Oversight, Agency Action, Federal Rights and F...",Ben Sasse (R-NE),Richard Blumenthal (D-CT),[14]
82,Rules and Administration,Rules and Administration,Roy Blunt (R-MO),Amy Klobuchar (D-MN),[1][2]
83,Small Business and Entrepreneurship,Small Business and Entrepreneurship,Marco Rubio (R-FL),Ben Cardin (D-MD),[1][2]


In [12]:
chairNamesCols = stripNames(chairNames)
rankingMembersCols = stripNames(rankingMembers)
len(importNames)
importNames = importNames.drop(importNames.index[0])
print(len(importNames))

84


In [13]:
importNames['Chair Names'] = chairNamesCols[0]
importNames['Party'] = chairNamesCols[1]
importNames['PartyAbbv'] = chairNamesCols[2]
importNames['State'] = chairNamesCols[3]
importNames['Ranking Members'] = rankingMembersCols[0]
importNames['Party'] = rankingMembersCols[1]
importNames['PartyAbbv'] = rankingMembersCols[2]
importNames['State'] = rankingMembersCols[3]

In [14]:
importNames['Committee'] = importNames['Committee'].replace(np.NaN, 'SubCommittee')
importNames = importNames.drop(columns=['Refs', 'Ranking Member', 'Chair', 'Party'])
importNames = importNames.rename(columns={'Committee.1':'SubComittee', 'PartyAbbv':'Party'})

In [15]:
importNames.to_csv('Chairs_RankingMembers.csv', index = False)